In [28]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# 1. Demand data (Material requirements of production orders)

In [29]:
# 1.1. Import the demand data

# Read the excel file
demand = pd.read_excel(io='input_template.xlsx', sheet_name = 'production_order')

# Filter the demand data to get only production orders with enough stock on-hand
demand = demand[demand['stock_check'] == 1]

# Check the data
print(demand.head())

  production_order item_code  material_requirements  stock_check
0              PO1       A57                  14219            1
1              PO1        A1                  12220            1
2              PO1        A7                   7582            1
3              PO2       A57                  15868            1
4              PO2        A6                  12000            1


In [30]:
# 1.2. Create lists of production orders and items

# Create lists of unique production orders and items
po_list = demand['production_order'].unique().tolist()
item_list_demand = demand['item_code'].unique().tolist()

# Check the data
print(f'''List of production orders: {po_list}''')
print(f'''List of materials: {item_list_demand}''')

List of production orders: ['PO1', 'PO2', 'PO3', 'PO4', 'PO5', 'PO6', 'PO7', 'PO8', 'PO9', 'PO10', 'PO11', 'PO12', 'PO13', 'PO14', 'PO15', 'PO16']
List of materials: ['A57', 'A1', 'A7', 'A6', 'A2', 'A58']


In [31]:
# 1.3. Combine (cross join) production orders and item lists

# Create a list to store combinations of production orders and items
po_item_list = []
for po in po_list:
    for item in item_list_demand:
        po_item_list.append((po, item))

# Convert the list to a dataframe
po_item_df = pd.DataFrame(po_item_list, columns=['production_order', 'item_code'])

# Check the data
print(po_item_df.head())

  production_order item_code
0              PO1       A57
1              PO1        A1
2              PO1        A7
3              PO1        A6
4              PO1        A2


In [32]:
# 1.4. Merge the combination dataframe with the demand data

# Merge the combination dataframe with the demand data to get the material requirements for each combination
po_item_df = po_item_df.merge(demand, on=['production_order', 'item_code'], how='left').replace(np.nan, 0)

# Drop stock_check column
po_item_df = po_item_df.drop(columns=['stock_check'])

# Change the data type of columns
po_item_df['material_requirements'] = po_item_df['material_requirements'].astype(int)

# Check the data
print(po_item_df.head())

  production_order item_code  material_requirements
0              PO1       A57                  14219
1              PO1        A1                  12220
2              PO1        A7                   7582
3              PO1        A6                      0
4              PO1        A2                      0


# 2. Supply data (Stock on-hand of bins)

In [33]:
# 2.1. Import the supply data

# Read the excel file
supply = pd.read_excel(io='input_template.xlsx', sheet_name = 'inventory')

# Check the data
print(supply)

      bin_no item_code  stock_on_hand
0          1        A1           2001
1          2        A1           2155
2          3        A1           2428
3          4        A1           2391
4          5        A1           2720
...      ...       ...            ...
1786    1787        A9           2209
1787    1788        A9           1640
1788    1789        A9           2259
1789    1790        A9           2583
1790    1791        A9           2088

[1791 rows x 3 columns]


In [34]:
# 2.2. Create lists of bins and items

# Create lists of unique bins and items
bin_list= supply['bin_no'].unique().tolist()
item_list_supply = supply['item_code'].unique().tolist()

# Check the data
print(f'''List of bins: {bin_list}''')
print(f'''List of items: {item_list_supply}''')

List of bins: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219

In [35]:
# 2.3. Combine (cross join) production order and item lists

# Create a list to store combinations of bins and items
bin_item = []
for bin_no in bin_list:
    for item in item_list_supply:
        bin_item.append((bin_no, item))

# Convert the list to a dataframe
bin_item_df = pd.DataFrame(bin_item, columns=['bin_no', 'item_code'])

# Check the data
print(bin_item_df.head())

   bin_no item_code
0       1        A1
1       1       A10
2       1       A11
3       1       A12
4       1       A13


In [36]:
# 2.4. Merge the combination dataframe with the supply data

# Merge the combination dataframe with the supply data to get the stock on-hand for each combination
bin_item_df = bin_item_df.merge(supply, on=['bin_no', 'item_code'], how='left').replace(np.nan, 0)

# Change the data type of columns
bin_item_df['stock_on_hand'] = bin_item_df['stock_on_hand'].astype(int)
bin_item_df['bin_no'] = bin_item_df['bin_no'].astype(str)

# Check the data
print(bin_item_df.head())

  bin_no item_code  stock_on_hand
0      1        A1           2001
1      1       A10              0
2      1       A11              0
3      1       A12              0
4      1       A13              0


# 3. Problem 1: Minimizing the number of bins allocated

In [37]:
# 3.1. Create the optimization model as a function for bin allocation

def bin_allocation_model(po_item_df, bin_item_df):

    # Sets
    I = po_item_df['production_order'].unique()  # Set of production orders
    J = po_item_df['item_code'].unique()  # Set of item codes
    K = bin_item_df['bin_no'].unique()  # Set of bins
    D = po_item_df.set_index(['production_order', 'item_code'])['material_requirements'].to_dict()  # Set of demand
    S = bin_item_df.set_index(['item_code', 'bin_no'])['stock_on_hand'].to_dict()  # Set of supply

    # Model
    bin_allocation_model = gp.Model('bin_allocation_optimization')

    # Variables
    x = bin_allocation_model.addVars(I, J, K, vtype=GRB.CONTINUOUS, name='the amount of item j from bin k to be allocated to production order i')
    y = bin_allocation_model.addVars(I, J, K, vtype=GRB.BINARY, name='the decision whether to allocate item j from bin k to production order i')

    # Objective function: Minimize the number of bins allocated
    bin_allocation_model.setObjective(gp.quicksum(y[i, j, k] for i in I for j in J for k in K), GRB.MINIMIZE)

    # Demand contraint: For each production order i and item j, the total amount allocated from all the bin must be greater than or equal to the demand
    for i in I:
        for j in J:
            bin_allocation_model.addConstr(gp.quicksum(x[i, j, k] for k in K) == D[i, j])

    # Supply constraint: For each item j and bin k, the total amount to be allocated to all the production orders must be less than or equal to the stock on hand
    for j in J:
        for k in K:
            bin_allocation_model.addConstr(gp.quicksum(x[i, j, k] for i in I) <= S[j, k])
            
    # Linking constraint: For each production order i, item j and bin k, if any amount of item j in bin k is allocated to production order i, 
    # the corresponding decision variable x[i, j, k] must be equal to 1
    for i in I:
        for j in J:
            for k in K:
                bin_allocation_model.addConstr(x[i, j, k] - S[j, k] * y[i, j, k] <= 0)

    # Run the model
    bin_allocation_model.optimize()

    # Results
    results_list_allocation = []
    for i in I:
        for j in J:
            for k in K:
                if y[i, j, k].X > 0:
                    results_list_allocation.append({
                        'production_order': i,
                        'item_code': j,
                        'bin_no': k,
                        'amount_allocated': x[i, j, k].X
                    })
    bin_allocation_result = pd.DataFrame(results_list_allocation)

    # Adjust the format of the result
    bin_allocation_decision = bin_allocation_result
    bin_allocation_decision['bin_no'] = bin_allocation_decision['bin_no'].astype(int)
    bin_allocation_decision['amount_allocated'] = bin_allocation_decision['amount_allocated'].astype(int)
    bin_allocation_decision = bin_allocation_decision.groupby(['bin_no', 'item_code','production_order']).agg(amount_allocated=('amount_allocated', 'sum'))
    bin_allocation_decision = bin_allocation_decision[bin_allocation_decision['amount_allocated'] > 0].sort_values(by=['bin_no', 'item_code', 'production_order'])

    return bin_allocation_result, bin_allocation_decision

In [38]:
# 3.2. Run the bin allocation model
bin_allocation_decision = bin_allocation_model(po_item_df, bin_item_df)[1]
bin_allocation_decision

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 182778 rows, 343872 columns and 518480 nonzeros
Model fingerprint: 0xb5230588
Variable types: 171936 continuous, 171936 integer (171936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 2e+04]
Presolve removed 181832 rows and 342368 columns
Presolve time: 0.06s
Presolved: 946 rows, 1504 columns, 3008 nonzeros
Variable types: 752 continuous, 752 integer (752 binary)
Found heuristic solution: objective 156.0000000

Root relaxation: objective 1.504340e+02, 328 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth 

,,,amount_allocated
bin_no,item_code,production_order,
1,A1,PO10,2001
2,A1,PO1,2155
3,A1,PO1,2428
4,A1,PO9,2391
5,A1,PO10,2720
...,...,...,...
1729,A7,PO1,2531
1730,A7,PO15,2550
1731,A7,PO10,2103


# 4. Problem 2: Minimizing the number of bins after combination

In [39]:
# 4.1. Merge the allocation result with supply data to check left over

# Get the result of bin allocation model for the input of bin combination model
bin_combination_df = bin_allocation_model(po_item_df, bin_item_df)[0].drop(columns=['production_order'])

# Group the dataframe to get the total amount allocated for each bin and item
bin_combination_df.groupby(by=['bin_no', 'item_code']).agg(amount_allocated=('amount_allocated', 'sum'))

# Change the data type of columns
bin_combination_df['bin_no'] = bin_combination_df['bin_no'].astype(str)

# Merge the dataframe with the supply data to calculate the left-over inventory
bin_combination_df = bin_combination_df.merge(bin_item_df, on=['bin_no', 'item_code'], how='left')
bin_combination_df['left_over_inventory'] = (bin_combination_df['stock_on_hand'] - bin_combination_df['amount_allocated']).astype(int)

# Capacity of the bins (this should be put into the raw data rather than here)
bin_combination_df['bin_storing_capacity'] = 3000

# Filter the dataframe to get only the bins that need to be combined (left-over inventory < 1000)
bin_combination_df['bin_need_to_combine'] = np.where((bin_combination_df['left_over_inventory'] < 1000) & ((bin_combination_df['amount_allocated'] > 0)), 1, 0)
bin_combination_df = bin_combination_df[bin_combination_df['bin_need_to_combine'] == 1]

# Sort the dataframe and drop unnecessary columns
bin_combination_df = bin_combination_df.sort_values(by=['bin_no', 'item_code'])[['bin_no', 'item_code', 'left_over_inventory', 'bin_storing_capacity']]

# Check the data
print(bin_combination_df.head())

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 182778 rows, 343872 columns and 518480 nonzeros
Model fingerprint: 0xb5230588
Variable types: 171936 continuous, 171936 integer (171936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 2e+04]
Presolve removed 181832 rows and 342368 columns
Presolve time: 0.06s
Presolved: 946 rows, 1504 columns, 3008 nonzeros
Variable types: 752 continuous, 752 integer (752 binary)
Found heuristic solution: objective 156.0000000

Root relaxation: objective 1.504340e+02, 328 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth 

In [40]:
# 4.2 Create list of items in bins that need to be combined

# Create a list of unique items that need to be combined
item_list_combination = bin_combination_df['item_code'].unique().tolist()

# Check the data
print(item_list_combination)

['A1', 'A57', 'A58', 'A6', 'A7', 'A2']


In [41]:
# 4.3. Create the optimization model as a function for bin combination

def bin_combination_model(df):

    # Create a dataframe to store all combinations of origin and destination bins with left-over inventory
    bin_combination_list = []
    for origin_bin in range(len(df)):
        for destination_bin in range(len(df)):
            bin_combination_list.append((df.iloc[origin_bin]['bin_no'], df.iloc[destination_bin]['bin_no']))
    bin_combination_df = pd.DataFrame(bin_combination_list, columns=['origin_bin', 'destination_bin']) 
    bin_combination_df = bin_combination_df.merge(df, left_on='origin_bin', right_on='bin_no', how='left')[['origin_bin', 'destination_bin', 'left_over_inventory']]

    # Sets
    M = bin_combination_df['origin_bin'].unique().tolist() # Set of origin bins
    N = bin_combination_df['destination_bin'].unique().tolist() # Set of destination bins
    Q = bin_combination_df.set_index(['origin_bin', 'destination_bin'])['left_over_inventory'].to_dict() # Set of left-over inventory of origin bins
    C = df.set_index(['bin_no'])['bin_storing_capacity'].to_dict() # Set of capacity of bins

    # Model
    bin_combination_model = gp.Model('bin_combination_optimization')

    # Variables
    a = bin_combination_model.addVars(M, N, vtype=GRB.BINARY, name='the decision whether to combine bin m to bin n or not')
    b = bin_combination_model.addVars(N, vtype=GRB.BINARY, name='the decision whether to use destination bin n or not')

    # Objective function: Minimizing the number of bins after combination
    bin_combination_model.setObjective(gp.quicksum(b[n] for n in N), GRB.MINIMIZE)

    # One-destination constraint: For each orgin bin, it must be allocate to only one destination bin
    for m in M:
        bin_combination_model.addConstr(gp.quicksum(a[m, n] for n in N) == 1)

    # Storing capacity constraint: For each destination bin, if it is used, 
    # the total amount of materials to be allocated from all origin bins to it must not exceed its storing capacity:
    for n in N:
        bin_combination_model.addConstr(gp.quicksum(a[m, n] * Q[m, n] for m in M) <= C[n] * b[n])

    # Linking constraint: For each origin bin and destination bin, if the origin bin is combined with the destination bin, 
    # the corresponding decision variable b[n] must be equal to 1, else 0
    for m in M:
        for n in N:
            bin_combination_model.addConstr(a[m, n] - b[n] <= 0)
    
    # The linking constraint can also be written as:
    # for n in N:
        # bin_combination_model.addConstr(quicksum(a[m, n] for m in M) <= len(M) * b[n])

    # Run the model
    bin_combination_model.optimize()

    # Results
    results_list_combination = []
    for m in M:
        for n in N:
            if a[m, n].X >= 0:
                results_list_combination.append({
                    'origin_bin': m,
                    'destination_bin': n,
                    'combine_decision': a[m, n].X,
                    'amount_combined': Q[m, n]
                })
    bin_combination_result = pd.DataFrame(results_list_combination)
    return bin_combination_result

In [42]:
# 4.4. Run the bin combination model for each material

# Create a list to store the combination results
bin_combination_decision = pd.DataFrame(columns=['origin_bin', 'destination_bin', 'combine_decision', 'amount_combined'])

# Run the bin combination model for each item
for item in item_list_combination:
    temp_df = bin_combination_df[bin_combination_df['item_code'] == item]
    bin_combination_result = bin_combination_model(temp_df)
    bin_combination_decision = pd.concat([bin_combination_decision, bin_combination_result], ignore_index=True)

# Filter the dataframe to get only combined bins
bin_combination_decision = bin_combination_decision[bin_combination_decision['combine_decision'] == 1]

# Merge the dataframe with the supply data to get the item code
bin_combination_decision = bin_combination_decision.merge(bin_combination_df, left_on='origin_bin', right_on='bin_no', how='left')[['origin_bin', 'destination_bin', 'amount_combined', 'item_code']]

# Check the data
print(bin_combination_decision.head())

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 783 rows, 756 columns and 2322 nonzeros
Model fingerprint: 0xed14d2f2
Variable types: 0 continuous, 756 integer (756 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 27 rows and 0 columns
Presolve time: 0.01s
Presolved: 756 rows, 756 columns, 2187 nonzeros
Variable types: 0 continuous, 756 integer (756 binary)

Root relaxation: objective 1.000000e+00, 730 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   G

C:\Users\thanh\AppData\Local\Temp\ipykernel_20092\127213142.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bin_combination_decision = pd.concat([bin_combination_decision, bin_combination_result], ignore_index=True)


Presolve time: 0.00s
Presolved: 58 rows, 84 columns, 168 nonzeros
Variable types: 0 continuous, 84 integer (84 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: objective 1.000000e+00, 57 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       1.0000000    1.00000  0.00%     -    0s

Explored 1 nodes (57 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 16 (of 16 available processors)

Solution count 3: 1 2 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up

# 5. Export the final results of 2 models to an excel file

In [43]:
# 5.1. Export the final results of 2 models to an excel file

# Export with 2 sheets with pivot table format
with pd.ExcelWriter('output_template.xlsx') as writer:
    bin_allocation_decision.to_excel(writer, sheet_name='bin_allocation_decision', index=True)
    bin_combination_decision.to_excel(writer, sheet_name='bin_combination_decision', index=True)